<a href="https://colab.research.google.com/github/victorgau/python_crawler/blob/master/4-3%20%E6%8A%93%E5%8F%96XML%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 抓取XML資料

提示：

使用 BeautifulSoup 也可以剖析 XML 檔喔！

```python
soup = BeautifulSoup(r.text, 'xml')
```

參考資料：

* [Wikipedia - XML](https://zh.wikipedia.org/wiki/XML)
* [高雄市公共腳踏車即時租賃站資訊](https://data.kcg.gov.tw/dataset/public-bike-rental/resource/1440e8fb-a87c-4eef-bdc7-9f5bc6a0481a)

高雄市 cbike ==> ubike !?

* https://www.c-bike.com.tw/Portal/zh-TW/News/Detail/1021

高雄市 YouBike 資料：

* https://kcg.youbike.com.tw/news/content?5ee727b91b99456b084eb624
* https://kcg.youbike.com.tw/station/2_map?5e0d965bbae27166a62a1993


## 抓取 cbike 資訊

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.c-bike.com.tw/xml/stationlistopendata.aspx"

In [3]:
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.text, 'xml')

In [5]:
stations = soup.select('Station')

In [6]:
stations[0]

<Station>
<StationID>1</StationID>
<StationNO>0001</StationNO>
<StationPic>http://www.c-bike.com.tw/Resources/Images/Upload/station/0001蓮池潭站_320x208.jpg</StationPic>
<StationPic2>http://www.c-bike.com.tw/Resources/Images/Upload/station/0001蓮池潭站_320x208.jpg</StationPic2>
<StationPic3>http://www.c-bike.com.tw/Resources/Images/Upload/station/0001蓮池潭站_320x208.jpg</StationPic3>
<StationMap/>
<StationName>蓮池潭站(關站)</StationName>
<StationAddress>高雄市左營區環潭路蓮池潭風景區管理所後方入口處。關站：109年6月17日</StationAddress>
<StationLat>22.680599</StationLat>
<StationLon>120.297466</StationLon>
<StationDesc>建設局風管所旁LotusPond（蓮池潭環潭路-停車場P）●無人自動租賃站來源引用-高雄市政府工務局關站：109年6月17日</StationDesc>
<StationNums1>0</StationNums1>
<StationNums2>32</StationNums2>
</Station>

In [7]:
import folium
import geocoder

In [8]:
# 使用 geocoder 取得特定住址的 GPS 座標
location = geocoder.osm('高雄市').latlng

In [9]:
m = folium.Map(location=location, zoom_start=16)

In [10]:
for station in stations:
    try:
        lat = float(station.select('StationLat')[0].text)
        lon = float(station.select('StationLon')[0].text)
        name = station.select('StationName')[0].text
        num1 = station.select('StationNums1')[0].text
        num2 = station.select('StationNums2')[0].text
        folium.Marker([lat, lon], popup="<div style='white-space:nowrap;text-align:center;'>{}<br>(可借：{}, 可停：{})</div>".format(name, num1, num2)).add_to(m)
    except Exception as e:
        print(e.args)

In [11]:
m

## 抓取 YouBike 資訊

In [12]:
url = "https://apis.youbike.com.tw/api/front/station/all?lang=tw&type=2"

In [13]:
data = requests.get(url).json()

In [14]:
# 使用 geocoder 取得特定住址的 GPS 座標
location = geocoder.osm('高雄市').latlng

In [15]:
m = folium.Map(location=location, zoom_start=16)

In [16]:
for station in data['retVal']:
    try:
        if station['area_code']=="12":
            lat = float(station['lat'])
            lng = float(station['lng'])
            name = station['name_tw']
            num1 = station['available_spaces']
            num2 = station['empty_spaces']
            folium.Marker([lat, lng], popup="<div style='white-space:nowrap;text-align:center;'>{}<br>(可借：{}, 可停：{})</div>".format(name, num1, num2)).add_to(m)
    except Exception as e:
        print(e.args)

In [17]:
m